In [1]:
import numpy as np
from scipy.stats import norm

rng = np.random.default_rng(7)

def ztest_pvalue(a_conv, a_n, b_conv, b_n):
    p_a = a_conv / a_n
    p_b = b_conv / b_n
    p_pool = (a_conv + b_conv) / (a_n + b_n)
    se = np.sqrt(p_pool * (1 - p_pool) * (1 / a_n + 1 / b_n))
    if se == 0:
        return 1.0, 0.0
    z = (p_b - p_a) / se
    p = 2 * (1 - norm.cdf(abs(z)))
    return p, z

def simulate_feasibility(
    p_base=0.05,
    rel_lift=0.20,          # 20% relative lift: 0.05 -> 0.06
    daily_users=80,         # total users/day
    horizons=(7, 14, 21, 28),
    traffic_multipliers=None,  # e.g., weekday/holiday effects (len=max(horizons))
    alpha=0.05,
    sims=5000
):
    p_treat = p_base * (1 + rel_lift)
    max_days = max(horizons)

    if traffic_multipliers is None:
        traffic_multipliers = np.ones(max_days)
    else:
        traffic_multipliers = np.array(traffic_multipliers, dtype=float)
        assert len(traffic_multipliers) >= max_days

    results = []
    for H in horizons:
        wins = losses = 0
        for _ in range(sims):
            a_conv = b_conv = 0
            a_n = b_n = 0

            for d in range(H):
                u = int(round(daily_users * traffic_multipliers[d]))
                a_u = u // 2
                b_u = u - a_u

                a_conv += rng.binomial(a_u, p_base)
                b_conv += rng.binomial(b_u, p_treat)
                a_n += a_u
                b_n += b_u

            pval, z = ztest_pvalue(a_conv, a_n, b_conv, b_n)

            if pval < alpha:
                if z > 0:
                    wins += 1   # significant B > A
                else:
                    losses += 1 # significant B < A (false harm signal)
        results.append((H, wins / sims, losses / sims))
    return results

# Example: uneven traffic + a "holiday dip" on day 7 
# weekdays (days 0-4): 1.0, weekend: 0.7, holiday dip: 0.5
traffic = [1.0, 1.0, 1.0, 1.0, 1.0, 0.7, 0.7, 0.5] + [1.0]*30

out = simulate_feasibility(
    p_base=0.05,
    rel_lift=0.20,
    daily_users=80,
    horizons=(7, 14, 21, 28),
    traffic_multipliers=traffic,
    alpha=0.05,
    sims=5000
)

print("Horizon(days) | P(sig win by horizon) | P(sig loss by horizon)")
for H, p_win, p_loss in out:
    print(f"{H:>12} | {p_win:>21.3f} | {p_loss:>20.3f}")

Horizon(days) | P(sig win by horizon) | P(sig loss by horizon)
           7 |                 0.067 |                0.006
          14 |                 0.106 |                0.004
          21 |                 0.132 |                0.003
          28 |                 0.172 |                0.002
